In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/yelp.csv", index_col=0)
data

In [3]:
data.info()

In [4]:
data['business_id'].nunique()

In [5]:
data['stars'].value_counts()

In [6]:
data['useful'].value_counts()

In [7]:
data[data['useful'] >500]

In [8]:
data[data['useful'] > 60]['stars'].value_counts()

In [9]:
data['funny'].value_counts()

In [10]:
data['cool'].value_counts()

In [11]:
data['text_len'] = data['text'].apply(len)

In [12]:
data

In [13]:
for content in data['text'][5000:5500]:
    print(content)

In [14]:
import string

In [15]:
string.punctuation

In [16]:
def remove_punc(x):
    cleaned = []
    for i in x:
        if i not in string.punctuation:
            cleaned.append(i)
    cleaned = "".join(cleaned)
    cleaned = cleaned.replace("   ", " ").replace("  ", " ").replace("\n", "").replace("\t", "").replace("\r", "")

    return cleaned
            

In [17]:
# test = '''DO NOT USE THIS SERVICE!!!!!!!!!!!!!! We have no idea how to take care of pools (this is our first) and so we trusted Scott and his company to take care of us. BAD IDEA. This has been our pool service for a year and we were on vacation and came home to find the pool level very low. We contacted Scott from Sky Blue and asked him what could be causing the low pool level and he replied rudely that " they dont manage water levels, thats all up to you." We asked him about our autofill feature and he said "theres no such thing". We knew he was full of s*&t right there because we may be ignorant on pool management but we do know that we have an autofill feature. We immediately hung up the phone and contacted a second opinion (a friend that works for corporate casino pool management) We have just found out that we are going to have to shell out over $1000 dollars to fix the incompetence of Sky Blue. Valves were turned the wrong way, the skimmer was shut off, and our pool lights are no longer working most likely because the ph levels in the pool were too acidic and water ate through. Also our filters pressures were through the roof. Most likely due to not being cleaned (the filters were only a year old so we know its not because they are old.)They are supposed to be cleaned every 3 months. There were multiple leaks. Also our salt cell system was mismanaged and has to be replaced. These are just SOME of the things found wrong.  We just assumed everything was fine with the pool. Never did Sky Blue let us know our filter system was about to explode or anything else for that matter! UGH!!! Good pool cleaning guys are hard to come by, especially in Vegas.Very sad. Its all good though, because I believe in Karma and Sky Blue has theirs coming :).'''

In [18]:
# remove_punc(test)

In [19]:
data['text'] = data['text'].apply(remove_punc)

In [20]:
data.head()

In [21]:
data['text'][:10]

In [22]:
sns.histplot(data['text_len'])

In [23]:
data[data['text_len'] > 4000]['stars'].value_counts()

In [24]:
data[data['text_len'] > 4500]['stars'].value_counts()

In [25]:
data[data['text_len'] < 1000]['stars'].value_counts()

In [26]:
y

# 상투적인 단어 제거하기
* nltk 라이브러리로 상투적인 언어(stopwords)제거하기

In [ ]:
# !pip install nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
print(stopwords.words('english'))

In [ ]:
data['text'].iloc[0]

In [ ]:
for i in data['text'].iloc[0].split():
    if i not in stopwords.words('english'):
        print(i)

In [ ]:
stopwords2 = stopwords.words('english')

In [ ]:
stopwords2.extend(['they', 'non', 'also'])

In [ ]:
stopwords2.extend(['they'])

In [ ]:
stopwords2

In [ ]:
def remove_stopwords(x):
    cleaned = []
    x = x.lower()
    for i in x.split():
        if i not in stopwords2:
            cleaned.append(i)
    cleaned = " ".join(cleaned)
    
    return cleaned

In [ ]:
data['text'] = data['text'].apply(remove_stopwords)
data['text'][:10]

In [ ]:
X = data[['text']]
y = data['stars']

In [ ]:
X

In [ ]:
y

# 자주 등장하는 단어 빈도 분석

In [ ]:
whole_words = []
for words in data['text']:
    for word in words.split():
        whole_words.append(word)

In [ ]:
len(whole_words)

In [ ]:
stopwords2.extend(['place', 'get'])

In [ ]:
result = []
for i in whole_words:
    if i not in stopwords2:
        result.append(i)

In [ ]:
from nltk.probability import FreqDist

In [ ]:
FreqDist(result)

In [ ]:
plt.figure(figsize=(20,10))
FreqDist(result).plot(50)

In [ ]:
result2 = " ".join(result)
print(len(result2))

# Word Could 만들기

In [ ]:
# !pip install wordcloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
wc = WordCloud().generate(result2)
plt.figure(figsize=(10,5))
plt.imshow(wc)
plt.axis('off')

# 평점 1점과 5점으로 데이터를 나누고 각각 word cloud 만들기

In [ ]:
data2 = pd.concat([X, y], axis=1)
data2

In [ ]:
good = data2.loc[data2['stars'] == 5, 'text'] 
good.head()

In [ ]:
bad = data2.loc[data2['stars'] == 1, 'text'] 
bad.head()

In [ ]:
good_text = " ".join(list(good))
len(good_text)

In [ ]:
bad_text = " ".join(list(bad))
len(bad_text)

In [ ]:
wc = WordCloud(stopwords= ['place', 'get', 'great', 'good'], max_words=2000, random_state=200).generate(good_text)
plt.figure(figsize=(10,5))
plt.imshow(wc)
plt.axis('off')

In [ ]:
wc = WordCloud(stopwords= ['place', 'get']).generate(bad_text)
plt.figure(figsize=(10,5))
plt.imshow(wc)
plt.axis('off')

In [ ]:
X

In [ ]:
y

# 나이브 베이즈를 이용해서 감성분석 하기

## text를 벡터화 하기  CountVectorizer
* 단어의 빈도를 세고 숫자로 변형

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()
X_vec = cv.fit_transform(X['text'])
X_vec

In [ ]:
for i in X_vec[:1]:
    print(i)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_vec, y, test_size=0.4, random_state=7)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=7)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
pred = mnb.predict(X_valid)
print(accuracy_score(y_valid, pred))
print(classification_report(y_valid, pred))

In [ ]:
test_pred = mnb.predict(X_test)
print(accuracy_score(y_test, test_pred))
print(classification_report(y_test, test_pred))

In [ ]:
# !pip install xgboost

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=500, max_depth=5, n_jobs= -1 , random_state=7)
rfc.fit(X_train, y_train)
pred = rfc.predict(X_valid)
print(accuracy_score(y_valid, pred))
print(classification_report(y_valid, pred))